In [48]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [49]:
prefix = "train"

In [50]:
df = pd.read_csv("./data/" + prefix + "/labelled.csv")

In [51]:
# take 100 sequential data points with the same label and transform into (100, 2, 3)
# 100 rows, 2 columns (gyro, accel), 3 dims (x, y, z)

train_df = pd.DataFrame()

# Augment training data
datapoints = 100
# Every n steps take the next 100 datapoints and add them as a row
n_steps = 25
# The ratio datapoints/n_steps is how much the data will be augmented

for i in range(df.index.start, df.index.stop, n_steps):
	if len(set(list(df.loc[i:i+datapoints-1]["label"]))) != 1:
		continue

	gyro_x = list(df.loc[i:i+datapoints-1]["x_gyro"])
	gyro_y = list(df.loc[i:i+datapoints-1]["y_gyro"])
	gyro_z = list(df.loc[i:i+datapoints-1]["z_gyro"])

	acce_x = list(df.loc[i:i+datapoints-1]["x"])
	acce_y = list(df.loc[i:i+datapoints-1]["y"])
	acce_z = list(df.loc[i:i+datapoints-1]["z"])

	new_df_dict = dict()
	for (col, name) in [(gyro_x, "gyro_x"), (gyro_y, "gyro_y"), (gyro_z, "gyro_z"), (acce_x, "acce_x"), (acce_y, "acce_y"), (acce_z, "acce_z")]:
		if len(col) == datapoints:
			for c in range(datapoints):
				new_df_dict[name + "_" + str(c)] = [col[c]]


	new_df_dict["longitude"] = [df.loc[i]["longitude"]]
	new_df_dict["latitude"] = [df.loc[i]["latitude"]]
	new_df_dict["label"] = [df.loc[i]["label"]]

	new_df = pd.DataFrame.from_dict(new_df_dict)
	
	train_df = pd.concat([train_df, new_df], ignore_index = True)

In [52]:
train_df["label"] = train_df["label"].map({"lift": 0, "piste": 1})

In [53]:
train_df = train_df.dropna()

In [54]:
train_df["label"].value_counts()

0    9246
1    3321
Name: label, dtype: int64

In [55]:
train_df.to_csv("./data/" + prefix + "/" + prefix + ".csv", index=False)